In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
# Load dataset
file_path = "heart_disease_uci.csv"  # Update path if needed
df = pd.read_csv(file_path)


In [12]:
# Drop unnecessary columns
df = df.drop(columns=['id', 'dataset'], errors='ignore')


In [13]:
# Convert target variable: presence of heart disease (num > 0 -> 1, else 0)
df['num'] = (df['num'] > 0).astype(int)

In [14]:
# Fill missing numerical values with the median
numerical_cols = ['trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())

In [15]:
# Encode categorical variables
from sklearn.preprocessing import LabelEncoder, StandardScaler
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col].astype(str))

In [17]:
#Split into features and target
X = df.drop(columns=['num'])
y = df['num']


In [19]:
# Split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Define ANN model
def build_ann():
    model = keras.Sequential([
        layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 83.04%
Confusion Matrix:
[[890  33]
 [153  21]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.96      0.91       923
           1       0.39      0.12      0.18       174

    accuracy                           0.83      1097
   macro avg       0.62      0.54      0.54      1097
weighted avg       0.78      0.83      0.79      1097



In [21]:
# Define CNN model
def build_cnn():
    model = keras.Sequential([
        layers.Reshape((X.shape[1], 1, 1), input_shape=(X.shape[1],)),
        layers.Conv2D(16, (3, 1), activation='relu', padding='same'),
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [22]:
# Define LSTM model
def build_lstm():
    model = keras.Sequential([
        layers.Reshape((X.shape[1], 1), input_shape=(X.shape[1],)),
        layers.LSTM(16, activation='relu', return_sequences=False),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [24]:
#Define Autoencoder model
def build_autoencoder():
    encoder = keras.Sequential([
        layers.Dense(8, activation='relu', input_shape=(X.shape[1],))
    ])
    decoder = keras.Sequential([
        layers.Dense(X.shape[1], activation='sigmoid')
    ])
    autoencoder = keras.Sequential([encoder, decoder])
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder


In [25]:
# Initialize models
ann = build_ann()
cnn = build_cnn()
lstm = build_lstm()
autoencoder = build_autoencoder()


C:\Users\bosss\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\bosss\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [26]:
# Train all models
ann.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)
cnn.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)
lstm.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)
autoencoder.fit(X_train, X_train, epochs=20, batch_size=16, verbose=0)

In [27]:
# Evaluate models
ann_acc = ann.evaluate(X_test, y_test, verbose=0)[1]
cnn_acc = cnn.evaluate(X_test, y_test, verbose=0)[1]
lstm_acc = lstm.evaluate(X_test, y_test, verbose=0)[1]

In [28]:
# Autoencoder evaluation (reconstruction loss)
autoencoder_loss = autoencoder.evaluate(X_test, X_test, verbose=0)

In [29]:
# Print results
print(f"ANN Accuracy: {ann_acc:.4f}")
print(f"CNN Accuracy: {cnn_acc:.4f}")
print(f"LSTM Accuracy: {lstm_acc:.4f}")
print(f"Autoencoder Loss: {autoencoder_loss:.4f}")


ANN Accuracy: 0.8315
CNN Accuracy: 0.8424
LSTM Accuracy: 0.7609
Autoencoder Loss: 0.7294
